In [ ]:

import numpy as np
import pandas as pd
import spacy

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')[0:10000]
test_df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')[0:10000]

In [ ]:
print(train_df['target'].value_counts())

In [ ]:
max_feature = 20000

X_train = train_df['question_text']
X_test = test_df['question_text']

y_train = train_df['target']


In [ ]:
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)
    tokens = [word.text for word in doc
            if not word.is_stop
                and not word.is_space
                and not word.is_digit
                and not word.is_punct]
    return ' '.join(tokens)

X_train = X_train.apply(lambda x: preprocess(x))
X_test = X_test.apply(lambda x: preprocess(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)

train_vectors = tfidf_vectorizer.fit_transform(X_train).toarray()
test_vectors = tfidf_vectorizer.transform(X_test).toarray()

In [ ]:
train_vectors.shape

In [ ]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(14900,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

history = model.fit(train_vectors,
                    y_train,
                    epochs=20,
                    batch_size=128,
                    )

In [ ]:
y_pred = model.predict(test_vectors)

In [ ]:
submission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')[0:10000]
submission['prediction'] = y_pred
submission.to_csv('submission.csv', index=False)